<a href="https://colab.research.google.com/github/lucas-pinto/NUIN443/blob/main/problemSets/ps12_RNN_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install torch

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim


## Problem Set 12: Computing with Recurrent Neural Networks
Our goal today is to simulate a firing rate recurrent neural network, and develop an intuition for how it behaves.

Let's start by implementing a standard RNN of $N$ neurons with a hyperbolic tangent nonlinearity. Written in matrix notation, the dynamics of this system are defined by the equation:

\begin{equation}
\tau \frac{dx}{dt} = -x + J\phi(x) + J^{in}u\\
\end{equation}

where $\phi(x) = \tanh(x)$ is our firing rate nonlinearity, $J$ is our $N\times N$ recurrent weight matrix where $J_{ij} \sim \mathcal{N}(0,\frac{g}{\sqrt{N}})$, $\tau$ is the membrane time constant, $u$ is time-varying external input, and $J^{in}$ is a set of weights by which that input drives our neurons, where we will take $J^{in}_{ij} \sim \mathcal{N}(0,w)$.

### Problem 1. Simluating a random RNN.
First, let's simulate a randomly connected RNN and see how it responds to a driving input.

<font color=violet>1a. Complete the code snippet below to simulate the dynamics of a randomly connected RNN with recurrent weight matrix $J$ and input weights $J^{in}$ defined as above, and network parameters defined by the entries of the params struct.</font>

In [ ]:
params = {'N': 1000,      # number of neurons in the network
          'tau': 10,      # "membrane" time constant
          'dt': 1,        # simulation timestep
          'g': 2,         # gain on recurrent weights
          'n_inputs': 2,  # number of input channels
          'w': 0.1}       # variance of input weights

In [ ]:
def simulate_RNN(params, u = np.zeros(1), T=5000, J = np.zeros(1)):

    N = params['N']
    tau = params['tau']
    dt = params['dt']
    g = params['g']
    n_inputs = params['n_inputs']

    if not J.any():
        J = ...

    W = ...

    if not u.any():
        u = np.zeros((n_inputs,T))
    else:
        T = u.shape[1]

    x = np.random.rand(N,1)
    rstore = np.zeros((T,N))

    for t in range(1,T):
        dxdt = ...
        rstore[t,:] = ...

    return rstore

<font color=violet>1b. To check your work, simulate the network for 5000 timesteps in two conditions: first, simulate the spontaneous dynamics of the network; second, simulate the dynamics in response to a 2d driving input of $\sin(2\pi t * 0.001)$ and $\cos(2\pi t*0.001)$. Pick 5 units from the network and plot their activity and their autocorrelation for 2000 lags, for both cases. In the case of the driven network, omit the first 1000 timesteps from your calculation of autocorrelation to allow some "burn-in" time.</font>

<font color=violet>1c. Now, let's look at how network dynamics change as a function of input amplitude. For your same network, plot the average autocorrelation across 50 random neurons in the model (again for 2000 lags) for a range of input magnitudes, until you see evidence of your input in your network's average autocorrelation. At what input magnitude does the network become entrained? Repeat using g=0.5 and g=5; what changes?</font>

### Problem 2. Training an RNN using Torch

In practice, you're likely going to find yourself using a package like PyTorch or Tensorflow for simulating or training RNNs. While we're not going to make you code up this training process from scratch, it's worth running the steps yourself to see what you can do with the results. We'll provide the model setup and training code to work from.

To start, let's define a task: we're going to drive our network with a pair of inputs:
\begin{equation}
u_1(t) = \sin(2\pi f_0 t + \phi) + \sin(2\pi\frac{f_0}{2} t + \phi_{d}) + \eta_1(t)\\
u_2(t) = \cos(2\pi f_0 t + \phi) + \cos(2\pi\frac{f_0}{2} t + \phi_{d}) + \eta_2(t)
\end{equation}

where $\phi$ and $\phi_d$ are randomly selected phase shifts and $\eta(t)$ is Gaussian noise. We want our network to predict the value of the first sin and cosine signals 5 timesteps into the future, and ignore the other terms.

\begin{equation}
f_1(t) = \sin(2\pi f_0 (t+5) + \phi)\\
f_2(t) = \cos(2\pi f_0 (t+5) + \phi)
\end{equation}

In [ ]:
import numpy as np

def generate_sine_cosine_data(seq_length, num_sequences, freq=10, time_shift=5):
    X = []
    y = []
    for _ in range(num_sequences):
        t = np.linspace(0, seq_length, num=seq_length)

        # create our target signal- a sine wave and a cosine wave at the provided frequency, starting with random phase.
        phi = np.random.rand(1) * freq
        sine_target = np.sin(2 * np.pi * freq * t + phi)
        cosine_target = np.cos(2 * np.pi * freq * t + phi)
        sequence_target = np.stack((sine_target, cosine_target), axis=1)

        # create some distractor inputs to the network.
        phi = np.random.rand(1) * freq
        sine_distractor = np.sin(2 * np.pi * freq/2 * t + phi)
        cosine_distractor = np.cos(2 * np.pi * freq/2 * t + phi)
        noise_term = np.random.randn(sequence_target.shape[0],sequence_target.shape[1])

        sequence_input = np.stack((sine_target + sine_distractor, cosine_target + cosine_distractor), axis=1) + noise_term

        X.append(sequence_input[:-time_shift])
        y.append(sequence_target[time_shift:])
    return np.array(X), np.array(y)

seq_length = 300
num_sequences = 100
X, y = generate_sine_cosine_data(seq_length, num_sequences)

The following code will initialize our RNN model; be sure you run it and the cell above to set things up.

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

        # optional: if you uncomment the following lines you can modify the initial
        # weights of the model to see how it affects learning:
#         weights = self.rnn.state_dict()
#         weights['weight_hh_l0'] = torch.randn(hidden_size, hidden_size) * 1/np.sqrt(hidden_size)
#         self.rnn.load_state_dict(weights)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out

input_size = 2
hidden_size = 500
output_size = 2
num_layers = 1

model = RNN(input_size, hidden_size, output_size, num_layers)


Torch initializes the RNN with no particular structure in its recurrent weight matrix. Let's take a look at its eigenvalues to confirm this.

<font color=violet>2a. We can access the recurrent weights from our model using the command `model.rnn.state_dict()['weight_hh_l0'].numpy()`. Using this, plot the real and imaginary components of the eigenvalues of the untrained weight matrix. Add a plot of a circle whose radius is the variance of the weights divided by $\sqrt(N)$, and confirm that this roughly encloses the eigenvalues.</font>

Now, run the following code to train your model on the task! The loss will never go completely to zero (and you might not see a clean drop if you look just at every 5th entry), but the network should still get better at the task over time.

In [ ]:
X_train = torch.tensor(X, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.float32)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
batch_size = 64

for epoch in range(num_epochs):
    permutation = torch.randperm(X_train.size(0))
    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train[indices], y_train[indices]

        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Finally, run this cell as a sanity check that training worked. Training an RNN is a bit stochastic, so if your network didn't converge on a solution, you can go back and run for more epochs, or run until the loss drops below a target value.

In [ ]:
# Generate a test dataset
X_test, y_test = generate_sine_cosine_data(seq_length, 100)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

with torch.no_grad():
    predictions = model(X_test).numpy()

# Plot the predictions
trial = 10
plt.figure(figsize=(10, 4))
plt.plot(y_test[trial, :, 0], label='Actual Sine')
plt.plot(predictions[trial, :, 0], label='Predicted Sine')
plt.legend()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(y_test[trial, :, 1], label='Actual Cosine')
plt.plot(predictions[trial, :, 1], label='Predicted Cosine')
plt.legend()
plt.show()


<font color=violet>2b. What did training do to our recurrent weight matrix? To find out, plot the real and imaginary components of the eigenvalues of the trained weight matrix; on the same plot, add the eigenvalues of the original untrained model and the circle you computed previously. What has changed? Which of these changes are most relevant to the task this network is trying to perform?</font>

<font color=violet>2c. Now, let's use our simulation code from problem 1 to look at the effect of this modified weight matrix on the network's dynamics. Modify your original `simulate_RNN` function to now take $J$ as an optional input, and pass it the weight matrix of the untrained or trained model. Drive both versions with 2d Gaussian white noise input and again look at the autocorrelation. Can you see a change in the dynamics of the trained network?</font>